<a href="https://colab.research.google.com/github/blindspotm/fine-tuning/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
</div>


# Task
Load the first 5 lines from "/content/chatgpt_raw.jsonl" and "/content/claude_raw.jsonl". Parse the JSON objects to inspect the keys and print the content of these entries, focusing on the instructions and responses to compare formatting and content depth. Finally, summarize the findings regarding formatting differences and content quality to recommend a base model for the master file.

## Load and Inspect Data

### Subtask:
Load the first 5 lines from the raw JSONL files and inspect their structure.


**Reasoning**:
Load the first 5 lines from the raw JSONL files and inspect their structure as requested.



In [ ]:
import json

# Load first 5 lines from chatgpt_raw.jsonl
chatgpt_entries = []
with open('/content/chatgpt_raw.jsonl', 'r') as f:
    for _ in range(5):
        line = f.readline()
        if not line: break
        chatgpt_entries.append(json.loads(line))

# Load first 5 lines from claude_raw.jsonl
claude_entries = []
with open('/content/claude_raw.jsonl', 'r') as f:
    for _ in range(5):
        line = f.readline()
        if not line: break
        claude_entries.append(json.loads(line))

# Inspect structure
if chatgpt_entries:
    print("ChatGPT Data Keys:", chatgpt_entries[0].keys())
else:
    print("ChatGPT file is empty or could not be read.")

if claude_entries:
    print("Claude Data Keys:", claude_entries[0].keys())
else:
    print("Claude file is empty or could not be read.")

print("\nChatGPT Entries (First 5):")
print(chatgpt_entries)

print("\nClaude Entries (First 5):")
print(claude_entries)

# Task
Validate the structure of every line in "/content/chatgpt_raw.jsonl" and "/content/claude_raw.jsonl" to ensure they contain the keys 'instruction', 'input', and 'output'. Concurrently, analyze the 'output' text to count specific 'AI tells' (e.g., 'As an AI', 'Certainly', 'Here is', 'limitations') and 'Marketing Frameworks' (e.g., 'ROAS', 'CPA', 'Contribution Margin', 'guardrails'). Report the validation status and compare the phrase frequencies between the two datasets to formulate a data merging strategy.

## Validate Structure and Keys

### Subtask:
Iterate through every line of 'chatgpt_raw.jsonl' and 'claude_raw.jsonl'. Verify that every entry is valid JSON and contains exactly the keys 'instruction', 'input', and 'output'. Report any malformed lines or missing keys.


**Reasoning**:
I will write a script to iterate through 'chatgpt_raw.jsonl' and 'claude_raw.jsonl', verifying that each line is valid JSON and contains the required keys ('instruction', 'input', 'output'), and then print a summary of valid lines and any errors found.



In [ ]:
import json

file_paths = ['/content/chatgpt_raw.jsonl', '/content/claude_raw.jsonl']
required_keys = {'instruction', 'input', 'output'}

for file_path in file_paths:
    print(f"Checking file: {file_path}")
    total_lines = 0
    valid_lines = 0
    json_errors = 0
    missing_key_errors = 0
    error_details = []

    try:
        with open(file_path, 'r') as f:
            for line_num, line in enumerate(f, 1):
                total_lines += 1
                line = line.strip()
                if not line:
                    continue

                try:
                    data = json.loads(line)
                    if not required_keys.issubset(data.keys()):
                        missing_key_errors += 1
                        missing = required_keys - data.keys()
                        error_details.append(f"Line {line_num}: Missing keys {missing}")
                    else:
                        valid_lines += 1
                except json.JSONDecodeError:
                    json_errors += 1
                    error_details.append(f"Line {line_num}: JSON Decode Error")

        print(f"  Total lines: {total_lines}")
        print(f"  Valid entries: {valid_lines}")
        print(f"  JSON errors: {json_errors}")
        print(f"  Missing key errors: {missing_key_errors}")
        if error_details:
            print("  Error Details (first 10):")
            for err in error_details[:10]:
                print(f"    {err}")
            if len(error_details) > 10:
                print(f"    ... and {len(error_details) - 10} more errors.")
        print("-" * 30)

    except FileNotFoundError:
        print(f"  Error: File {file_path} not found.")


**Reasoning**:
I will script a content analysis to count specific 'AI tells' and 'Marketing Frameworks' phrases within the 'output' field of valid JSON entries in both files. This quantitative comparison will help determine the quality of the generated text and inform the data merging strategy.



In [ ]:
import json

ai_tells = ['As an AI', 'Certainly', 'Here is', 'limitations']
marketing_frameworks = ['ROAS', 'CPA', 'Contribution Margin', 'guardrails']

file_paths = ['/content/chatgpt_raw.jsonl', '/content/claude_raw.jsonl']
results = {}

for file_path in file_paths:
    file_name = file_path.split('/')[-1]
    results[file_name] = {
        'AI Tells': {phrase: 0 for phrase in ai_tells},
        'Marketing Frameworks': {phrase: 0 for phrase in marketing_frameworks},
        'Total Outputs Analyzed': 0
    }

    try:
        with open(file_path, 'r') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    if 'output' in data:
                        text = data['output']
                        results[file_name]['Total Outputs Analyzed'] += 1

                        # Count AI Tells (case-insensitive)
                        for phrase in ai_tells:
                            if phrase.lower() in text.lower():
                                results[file_name]['AI Tells'][phrase] += 1

                        # Count Marketing Frameworks (case-insensitive)
                        for phrase in marketing_frameworks:
                            if phrase.lower() in text.lower():
                                results[file_name]['Marketing Frameworks'][phrase] += 1
                except json.JSONDecodeError:
                    continue # Skip malformed lines
    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Display Results
print("Phrase Frequency Analysis:")
for file_name, stats in results.items():
    print(f"\nFile: {file_name}")
    print(f"  Valid Entries Analyzed: {stats['Total Outputs Analyzed']}")
    print("  AI Tells:")
    for phrase, count in stats['AI Tells'].items():
        print(f"    - '{phrase}': {count}")
    print("  Marketing Frameworks:")
    for phrase, count in stats['Marketing Frameworks'].items():
        print(f"    - '{phrase}': {count}")

In [ ]:
import json

input_files = ['/content/chatgpt_raw.jsonl', '/content/claude_raw.jsonl']
output_file = '/content/merged_data.jsonl'

merged_data = []
total_errors = 0

print("Starting merge process...")

for file_path in input_files:
    print(f"Processing {file_path}...")
    with open(file_path, 'r') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line: continue

            try:
                entry = json.loads(line)
                # validation check (optional, based on previous step)
                if all(k in entry for k in ('instruction', 'input', 'output')):
                    merged_data.append(entry)
            except json.JSONDecodeError:
                total_errors += 1
                # Silently skip errors as per strategy, or print if needed
                pass

# Save the merged data
with open(output_file, 'w') as f:
    for entry in merged_data:
        f.write(json.dumps(entry) + '\n')

print("-" * 30)
print(f"Merge complete.")
print(f"Total valid entries merged: {len(merged_data)}")
print(f"Total lines skipped due to errors: {total_errors}")
print(f"Merged file saved to: {output_file}")

### Findings and Data Merging Strategy

**Validation Status:**
- **ChatGPT Data**: Found 3 malformed lines (JSON Decode Errors) out of 40 total lines. 37 entries are valid.
- **Claude Data**: All 38 lines are valid JSON entries.

**Content Analysis:**
- **AI Tells**: Both datasets are clean, showing 0 instances of common AI refusal or conversational fillers like "As an AI".
- **Domain Relevance**: Both datasets successfully incorporate marketing frameworks. ChatGPT has a slightly higher density of "CPA" and "guardrails" references, while Claude emphasizes "ROAS".

**Merging Strategy:**
1. **Filter**: During the merge process, explicitly handle `json.JSONDecodeError` to skip the 3 corrupt lines in `chatgpt_raw.jsonl`.
2. **Combine**: Concatenate the 37 valid ChatGPT entries with the 38 Claude entries.
3. **Post-Processing**: Since "AI tells" are absent, no aggressive text filtering is required for these specific phrases.

### Findings and Data Merging Strategy

**Validation Status:**
- **ChatGPT Data**: Found 3 malformed lines (JSON Decode Errors) out of 40 total lines. 37 entries are valid.
- **Claude Data**: All 38 lines are valid JSON entries.

**Content Analysis:**
- **AI Tells**: Both datasets are clean, showing 0 instances of common AI refusal or conversational fillers like "As an AI".
- **Domain Relevance**: Both datasets successfully incorporate marketing frameworks. ChatGPT has a slightly higher density of "CPA" and "guardrails" references, while Claude emphasizes "ROAS".

**Merging Strategy:**
1. **Filter**: During the merge process, explicitly handle `json.JSONDecodeError` to skip the 3 corrupt lines in `chatgpt_raw.jsonl`.
2. **Combine**: Concatenate the 37 valid ChatGPT entries with the 38 Claude entries.
3. **Post-Processing**: Since "AI tells" are absent, no aggressive text filtering is required for these specific phrases.

### Findings and Data Merging Strategy

**Validation Status:**
- **ChatGPT Data**: Found 3 malformed lines (JSON Decode Errors) out of 40 total lines. 37 entries are valid.
- **Claude Data**: All 38 lines are valid JSON entries.

**Content Analysis:**
- **AI Tells**: Both datasets are clean, showing 0 instances of common AI refusal or conversational fillers like "As an AI".
- **Domain Relevance**: Both datasets successfully incorporate marketing frameworks. ChatGPT has a slightly higher density of "CPA" and "guardrails" references, while Claude emphasizes "ROAS".

**Merging Strategy:**
1. **Filter**: During the merge process, explicitly handle `json.JSONDecodeError` to skip the 3 corrupt lines in `chatgpt_raw.jsonl`.
2. **Combine**: Concatenate the 37 valid ChatGPT entries with the 38 valid Claude entries.
3. **Post-Processing**: Since "AI tells" are absent, no aggressive text filtering is required for these specific phrases.

## Summary:

### Q&A

**Q: What is the validation status of the provided JSONL datasets?**
A: The `/content/claude_raw.jsonl` file is perfectly structured, with all 38 lines being valid JSON entries containing the required keys. However, `/content/chatgpt_raw.jsonl` contains data corruption: out of 40 total lines, 3 lines triggered JSON decode errors, resulting in 37 valid entries.

**Q: What is the recommended data merging strategy based on the content analysis?**
A: The analysis suggests a "Filter and Combine" strategy. Specifically, the 3 malformed lines in the ChatGPT dataset should be discarded using exception handling. The remaining 37 valid ChatGPT entries should be concatenated with the 38 Claude entries to form a final dataset of 75 examples. No aggressive text filtering is needed for "AI tells" as none were detected.

### Data Analysis Key Findings

*   **Structure Validation**:
    *   **ChatGPT Data**: 37 out of 40 lines (92.5%) are valid. 3 lines failed due to `json.JSONDecodeError`.
    *   **Claude Data**: 38 out of 38 lines (100%) are valid and contain the required keys ('instruction', 'input', 'output').
*   **Phrase Frequency - AI Tells**:
    *   Both datasets are devoid of common AI refusal or conversational filler phrases (e.g., "As an AI", "Certainly", "Here is"), showing a count of 0 for all tracked terms.
*   **Phrase Frequency - Marketing Frameworks**:
    *   Both datasets successfully integrate domain specific terms. The ChatGPT data shows a higher prevalence of "CPA" and "guardrails", whereas the Claude data favors "ROAS".

### Insights or Next Steps

*   **Pipeline Adjustment**: The data ingestion pipeline for ChatGPT outputs requires investigation to identify the source of the JSON corruption affecting 7.5% of the data.
*   **Dataset Consolidation**: Proceed immediately with merging the 75 clean records, as the absence of "AI tells" implies the text data is high-quality and ready for downstream tasks without further cleaning.


# Task
Add a text cell to explicitly explain the 'JSON Decode Error' logs observed during the merge process. Clarify that 3 lines in the ChatGPT dataset were malformed and skipped, and confirm that the merge process successfully completed with the remaining 75 entries stored in "/content/merged_data.jsonl".

## Clarify Data Validation Results

### Subtask:
Add a text cell to explain the validation errors and successful merge.


## Summary:

### Q&A
**Q: What caused the 'JSON Decode Error' logs during the merge process?**
A: The errors were caused by 3 specific lines in the ChatGPT dataset that were malformed. These lines were identified and skipped to prevent the process from failing.

**Q: Was the merged file successfully created?**
A: Yes, the merge process was completed successfully. The resulting file contains the remaining 75 valid entries and is stored in `/content/merged_data.jsonl`.

### Data Analysis Key Findings
*   **Error Identification:** The validation process detected **3** malformed lines in the ChatGPT input data that resulted in JSON decode errors.
*   **Data Preservation:** These problematic lines were skipped, ensuring the remaining data was not corrupted.
*   **Final Output:** The final merged dataset was successfully saved to `/content/merged_data.jsonl` with **75** verified entries.

### Insights or Next Steps
*   The dataset is now clean and technically validated, making it safe for downstream tasks such as statistical analysis or model training.
*   Future data collection for the ChatGPT source should be monitored for JSON formatting consistency to reduce data loss.


# Task
Load the `/content/merged_data.jsonl` dataset using the Hugging Face `datasets` library. Define the Alpaca prompt template and a formatting function to structure the 'instruction', 'input', and 'output' fields, ensuring the EOS token is added. Apply this formatting to the dataset to prepare it for training.

## Load and Format Data

### Subtask:
Load the merged dataset and prepare it for training by applying the Alpaca formatting template.


**Reasoning**:
Load the merged dataset, define the Alpaca formatting function with the tokenizer's EOS token, and apply the formatting to prepare the data for training.



In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="/content/merged_data.jsonl", split="train")

# Define the Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Retrieve the EOS_TOKEN from the tokenizer (assumed to be defined in previous cells)
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Format the prompt and append the EOS_TOKEN
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched = True)

**Reasoning**:
Verify the formatting by printing the first element of the processed dataset to ensure the 'text' field is correctly structured with the Alpaca template and EOS token.



In [ ]:
print(dataset[0]["text"])

## Execute Training

### Subtask:
Initialize the SFTTrainer with the prepared dataset and training parameters, then run the training process.


**Reasoning**:
Initialize the SFTTrainer with the specified configuration and dataset, then execute the training process.



In [ ]:
from trl import SFTConfig, SFTTrainer
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

**Reasoning**:
The previous execution produced excessive output which might have been flagged as an error or interruption. I will provide the correct code to initialize the trainer and run the training, ensuring all parameters are set as specified in the instructions.



In [ ]:
from trl import SFTConfig, SFTTrainer
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

**Reasoning**:
I will initialize the SFTTrainer with the specific hyperparameters provided in the instructions (batch size, learning rate, max steps, etc.) and the formatted dataset, then execute the training process to fine-tune the model.



In [ ]:
from trl import SFTConfig, SFTTrainer
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

**Reasoning**:
I will initialize the SFTTrainer with the specific hyperparameters provided in the instructions (batch size, learning rate, max steps, etc.) and the formatted dataset, then execute the training process to fine-tune the model.



In [ ]:
from trl import SFTConfig, SFTTrainer
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

# Task
List the contents of the "/content/outputs/checkpoint-60" directory to verify that files such as `adapter_model.safetensors` and `trainer_state.json` exist. Then, load the `trainer_state.json` file to create a line plot of the training loss over the recorded steps. Finally, summarize the training status and confirm if the model is ready for inference based on the presence of the checkpoint files and the loss curve.

## Verify Training Completion

### Subtask:
List the contents of the output directory to verify that critical checkpoint files exist.


**Reasoning**:
I will verify the existence of critical training artifacts in the specified checkpoint directory to confirm that the training process successfully saved the model and state.



In [ ]:
import os

output_dir = "/content/outputs/checkpoint-60"

if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    print(f"Files in {output_dir}:\n{files}\n")

    required_files = ["adapter_model.safetensors", "trainer_state.json"]
    for file in required_files:
        if file in files:
            print(f"CONFIRMED: {file} exists.")
        else:
            print(f"MISSING: {file} not found.")
else:
    print(f"Directory not found: {output_dir}")

**Reasoning**:
I will load the 'trainer_state.json' file, extract the training loss history, and plot the loss curve over the training steps to visualize the model's convergence.



In [ ]:
import json
import matplotlib.pyplot as plt

trainer_state_path = "/content/outputs/checkpoint-60/trainer_state.json"

with open(trainer_state_path, 'r') as f:
    data = json.load(f)

log_history = data.get("log_history", [])

steps = []
losses = []

for entry in log_history:
    if "loss" in entry and "step" in entry:
        steps.append(entry["step"])
        losses.append(entry["loss"])

if steps and losses:
    plt.figure(figsize=(10, 6))
    plt.plot(steps, losses, marker='o', linestyle='-', color='b')
    plt.title("Training Loss Over Steps")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.show()
else:
    print("No loss data found in log_history.")

### Training Summary

*   **Checkpoint Verification**: Successfully validated the presence of `adapter_model.safetensors` and `trainer_state.json` in the `/content/outputs/checkpoint-60` directory.
*   **Training Loss**: The loss curve demonstrates a consistent downward trend, decreasing from an initial value of approximately 0.26 to around 0.04 by step 60. This indicates that the model has effectively converged and learned from the dataset.
*   **Status**: The model fine-tuning is complete, the artifacts are saved, and the model is ready for inference.

**Reasoning**:
I will verify the existence of critical training artifacts in the specified checkpoint directory to confirm that the training process successfully saved the model and state.



In [ ]:
import os

output_dir = "/content/outputs/checkpoint-60"

if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    print(f"Files in {output_dir}:\n{files}\n")

    required_files = ["adapter_model.safetensors", "trainer_state.json"]
    for file in required_files:
        if file in files:
            print(f"CONFIRMED: {file} exists.")
        else:
            print(f"MISSING: {file} not found.")
else:
    print(f"Directory not found: {output_dir}")

## Inference

### Subtask:
Run the fine-tuned model to generate a response for a test input.


**Reasoning**:
Enable inference mode on the model, prepare the test input using the alpaca prompt template, tokenize it, generate a response, and then decode and print the output.



In [ ]:
# Save the LoRA adapters and tokenizer locally
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

print("Model adapters and tokenizer saved to 'lora_model' directory.")

In [ ]:
from unsloth import FastLanguageModel

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define input using the alpaca_prompt format
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# Generate response
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

# Decode and print
print(tokenizer.batch_decode(outputs))

## Final Task

### Subtask:
Summarize the verification results and confirm the model is ready for inference.


## Summary:

### Q&A
**Q:** Are the required checkpoint files available for inference?
**A:** Yes, the contents of the `/content/outputs/checkpoint-60` directory were verified, confirming the presence of essential files such as `adapter_model.safetensors` and `trainer_state.json`.

**Q:** Did the model training converge successfully?
**A:** Yes, the analysis of the training log history indicates successful convergence, as the loss consistently decreased over the recorded steps.

### Data Analysis Key Findings
*   **Artifact Verification**: The output directory `/content/outputs/checkpoint-60` contains all necessary model artifacts, including `adapter_model.safetensors`, `trainer_state.json`, `tokenizer.json`, and optimizer states.
*   **Loss Analysis**: The training loss demonstrated a clear downward trajectory, starting at approximately **0.26** and dropping to around **0.04** by step 60, indicating the model effectively learned from the training data.
*   **Functional Inference**: The model successfully processed a test prompt regarding the Fibonacci sequence ("1, 1, 2, 3, 5, 8"), correctly generating the next number in the sequence (**13**) and providing a coherent textual explanation.

### Insights or Next Steps
*   The consistent drop in loss combined with the successful inference test confirms that the fine-tuning process was stable and the model is technically sound.
*   The model is now ready for more extensive testing; future steps should involve evaluating the model against a diverse set of prompts to ensure robustness beyond the initial validation case.


# Task
Save the fine-tuned model to GGUF format using `model.save_pretrained_gguf` with the `q4_k_m` quantization method, storing it in the "model" directory. Then, list the contents of the "model" directory to verify the presence of the `.gguf` file. Finally, confirm the conversion is complete.

## Convert to GGUF

### Subtask:
Save the model in GGUF format with q4_k_m quantization.


**Reasoning**:
I will execute the command to save the model in GGUF format with q4_k_m quantization as requested in the instructions.



In [ ]:
# Save the model in GGUF format with q4_k_m quantization
model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")

# Task
List the contents of the "model" directory to verify that the GGUF file has been successfully created. Finally, print a confirmation message that the GGUF model is ready for use.
# Plan:
* **Verify GGUF Output**: List the contents of the "model" directory to confirm that the `.gguf` file exists. This verifies that the conversion and saving process completed successfully.
* **Final Task**: Confirm the GGUF model is ready for use.

## Verify GGUF Output

### Subtask:
Verify that the GGUF model file was successfully created in the 'model' directory.


**Reasoning**:
I will verify the creation of the GGUF file by listing the contents of the 'model' directory and checking for any file with the '.gguf' extension.



# Task
Mount the user's Google Drive to `/content/drive`. Create a backup folder named `DigitalStrategist_Backup` in the drive. Copy the `/content/merged_data.jsonl` file, the `/content/lora_model` directory, and the `/content/model` directory into the backup folder. Finally, confirm the backup is complete and provide a brief instruction on how to load these files back into the environment when resuming work.

## Mount Google Drive

### Subtask:
Mount the user's Google Drive to `/content/drive` to enable persistent file storage.


**Reasoning**:
I will mount the Google Drive to allow access to persistent storage as requested.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reasoning**:
I will create a backup directory on Google Drive and copy the specified files and directories into it to ensure data persistence.



In [ ]:
import shutil
import os

# Define source paths
source_file = '/content/merged_data.jsonl'
source_dirs = ['/content/lora_model', '/content/model']

# Define destination path
backup_dir = '/content/drive/My Drive/DigitalStrategist_Backup'

# Create backup directory if it doesn't exist
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)
    print(f"Created backup directory: {backup_dir}")
else:
    print(f"Backup directory already exists: {backup_dir}")

# Copy file
dest_file = os.path.join(backup_dir, os.path.basename(source_file))
shutil.copy2(source_file, dest_file)
print(f"Copied {source_file} to {dest_file}")

# Copy directories
for source_dir in source_dirs:
    dir_name = os.path.basename(source_dir)
    dest_dir = os.path.join(backup_dir, dir_name)
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)  # Remove existing backup of the directory to ensure clean copy
    shutil.copytree(source_dir, dest_dir)
    print(f"Copied {source_dir} to {dest_dir}")

print("\nBackup complete!")
print("To load these files back in a future session:")
print("1. Mount Google Drive.")
print(f"2. Copy files from '{backup_dir}' back to '/content/'.")

In [ ]:
import json

# Replace with your notebook's name
filename = "Llama3.1_(8B)-Alpaca.ipynb"

with open(filename, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Delete the problematic widgets metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("Fixed: Removed problematic widget metadata.")

with open(filename, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

# Task
List the files and directories in `/content/drive/My Drive/DigitalStrategist_Backup` to verify the backup status. After listing, report which files are safely backed up and which might need to be re-copied.

## Verify Backup Contents

### Subtask:
List the files and directories in the backup folder to assess the integrity of the backup following the previous interruption.


**Reasoning**:
I will write a script to inspect the backup directory on Google Drive, listing files and checking the contents of subdirectories to determine which parts of the backup were successful before the interruption.



In [ ]:
import os

backup_dir = '/content/drive/My Drive/DigitalStrategist_Backup'

print(f"Checking backup directory: {backup_dir}")

if os.path.exists(backup_dir):
    contents = os.listdir(backup_dir)
    print(f"Contents of backup directory:\n{contents}\n")

    # Check for specific items
    expected_items = ['merged_data.jsonl', 'lora_model', 'model']
    for item in expected_items:
        item_path = os.path.join(backup_dir, item)
        if item in contents:
            if os.path.isdir(item_path):
                sub_contents = os.listdir(item_path)
                print(f"Found directory '{item}' with {len(sub_contents)} files:")
                print(sub_contents)
            else:
                print(f"Found file: {item}")
        else:
            print(f"MISSING: {item}")
else:
    print(f"Backup directory not found at: {backup_dir}")

In [ ]:
import requests

# This fetches the name of the current notebook session
filename = requests.get("http://172.28.0.12:9000/api/sessions").json()[0]["name"]

print(f"Your notebook name is: {filename}")

In [ ]:
import json

# Replace with your notebook's name
filename = "Llama3.1_(8B)-Alpaca.ipynb"

with open(filename, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Delete the problematic widgets metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("Fixed: Removed problematic widget metadata.")

with open(filename, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

In [ ]:
import os
import json
import requests

# 1. Automatically find the name of the notebook you are currently in
try:
    # Use the Colab internal API to get the current notebook filename
    filename = requests.get("http://172.28.0.12:9000/api/sessions").json()[0]["name"]
    if not filename.endswith(".ipynb"):
        filename += ".ipynb"
except:
    # Fallback: If you renamed it manually, it might be in /content/
    filename = "Llama3.1_(8B)-Alpaca.ipynb"

print(f"Targeting notebook: {filename}")

# 2. Check if the file exists in the local directory.
# If not, we have to save a local copy first to "edit" it.
if not os.path.exists(filename):
    print("Notebook file not found locally. Saving a temporary copy...")
    # This creates a physical file of your current progress so the script can edit it
    from google.colab import _message
    _message.blocking_request('save_ipynb', request={'path': filename}, timeout_sec=30)

# 3. Clean the metadata
with open(filename, "r", encoding="utf-8") as f:
    nb = json.load(f)

if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("Success: Removed problematic 'widgets' metadata.")
else:
    print("Metadata already clean or 'widgets' key not found.")

# 4. Save the 'Fixed' version
with open(filename, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

print(f"DONE! You can now download '{filename}' from the folder icon on the left and upload it to GitHub.")

In [ ]:
import os
import json
from google.colab import _message

# 1. Force Colab to save the current notebook to the local disk
filename = "Digital_Strategist_Final.ipynb"
print(f"Saving current progress to {filename}...")

# This command physically creates the file in your /content/ folder
_message.blocking_request('save_ipynb', request={'path': filename}, timeout_sec=30)

# 2. Open the file we just created and strip the broken 'widgets' metadata
if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        nb = json.load(f)

    # Check for the problematic 'widgets' key
    if "widgets" in nb.get("metadata", {}):
        del nb["metadata"]["widgets"]
        print("✅ Success: Problematic 'widgets' metadata removed.")
    else:
        print("ℹ️ Metadata was already clean.")

    # 3. Save the fixed version back to the disk
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(nb, f, indent=2)

    print(f"🚀 DONE! '{filename}' is ready.")
else:
    print("❌ Error: Could not create the local file. Try the 'Manual Fix' below.")